## Importando bibliotecas python

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRFRegressor


import mlflow
from mlflow.models.signature import infer_signature

## Lendo os dados

In [2]:
csv_path = "../data/processed/casas.csv"

df = pd.read_csv(csv_path)
df.head()

,tamanho,ano,garagem,preco
0,159.0,2003,2,208500
1,117.0,1976,2,181500
2,166.0,2001,2,223500
3,160.0,1915,3,140000
4,204.0,2000,3,250000


In [3]:
X = df.drop(columns="preco")
y = df.preco

print(f"O shape dos dados inicialmente: \n - X: {X.shape} \n - y: {y.shape}")
print("\n")

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2)
print(f"O shape dos dados de treino: \n - X: {X_train.shape} \n - y: {y_train.shape}")
print("\n")
print(f"O shape dos dados de teste : \n - X: {X_test.shape} \n - y: {y_test.shape}")

O shape dos dados inicialmente: 
 - X: (1460, 3) 
 - y: (1460,)


O shape dos dados de treino: 
 - X: (1168, 3) 
 - y: (1168,)


O shape dos dados de teste : 
 - X: (292, 3) 
 - y: (292,)


## Aplicando Modelo de Linear Regression

O objetivo aqui será criar um modelo simples para termos um base line.

In [4]:
mlflow.set_experiment("preco-casas-eda")

<Experiment: artifact_location='file:///home/carlos/Documentos/programas/Alura_/CD4ML/mlflow/notebooks/mlruns/767021303406655053', creation_time=1746796522623, experiment_id='767021303406655053', last_update_time=1746796522623, lifecycle_stage='active', name='preco-casas-eda', tags={}>

#### Definindo uma funcao para calcular as metricas de avaliacao do modelo

In [5]:
def metricas_regressao(y_true,y_pred):
    """retorna e printa as metricas de avaliacao para um modelo de regressao

    Args:
        y_true: Valores reais de y
        y_pred: Valores preditos de y

    Returns:
        mae, mse, rmse, r_2: mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score
    """
    mae = mean_absolute_error(y_true,y_pred)
    mse = mean_squared_error(y_true,y_pred)
    rmse = (mse)**(1/2)
    r_2 = r2_score(y_true,y_pred)
    
    print(f"Erro Medio Absoluto: {mae:.3f}")
    print(f"Erro Quadratico Medio: {mse:.3f}")
    print(f"Raiz do Erro Quadratico Medio: {rmse:.3f}")
    print(f"R quadrado: {r_2:.3f}")
    return mae, mse, rmse, r_2

In [6]:
import mlflow.sklearn


with mlflow.start_run():
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    y_pred = lin_reg.predict(X_test)
    
    sig_pred = lin_reg.predict(X_train[:5])
    signature = infer_signature(X_train[:5], sig_pred)
    
    mlflow.sklearn.log_model(lin_reg, 
                             'lin_reg',
                             signature=signature,
                             input_example=X_train[:5])
    
    mae, mse, rmse, r_2 = metricas_regressao(y_test,y_pred)
    mlflow.log_metric('mae',mae)
    mlflow.log_metric('mse',mse)
    mlflow.log_metric('rmse',rmse)
    mlflow.log_metric('r_2',r_2)
    


/home/carlos/anaconda3/envs/tensorflow-gpu/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Erro Medio Absoluto: 31146.875
Erro Quadratico Medio: 2450536214.058
Raiz do Erro Quadratico Medio: 49502.891
R quadrado: 0.635


#### Aplicando o XGBoost

In [ ]:
import mlflow.xgboost


with mlflow.start_run():    
    xgb = XGBRFRegressor(random_state=8293)
    xgb.fit(X_train, y_train)
    xgb_pred = xgb.predict(X_test)
    
    sig_pred = xgb.predict(X_train[:5])
    signature = infer_signature(X_train[:5],sig_pred)
    
    mlflow.xgboost.log_model(xgb,
                             'xgboost',
                             signature=sig_pred,
                             input_example=X[:5])
    
    mae, mse, rmse, r_2 = metricas_regressao(y_test, xgb_pred)
    mlflow.log_metric('mae',mae)
    mlflow.log_metric('mse',mse)
    mlflow.log_metric('rmse',rmse)
    mlflow.log_metric('r_2',r_2)
    

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=None, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=8293, reg_alpha=None, ...)

In [23]:
xgb_pred = xgb.predict(X_test)

In [24]:
metricas_regressao(y_test,xgb_pred)

Erro Medio Absoluto: 27754.246
Erro Quadratico Medio: 1738928128.000
Raiz do Erro Quadratico Medio: 41700.457
R quadrado: 0.760


(27754.24609375, 1738928128.0, 41700.45716775776, 0.759798526763916)